

## Objectives of this Notebook
In this notebook, I will download reduced DataFrames that consist of the columns deemed most important by a simple decision tree model, along with aggregated features of the columns considered least informative by the model. The goals of this feature engineering process are as follows:

1. **Feature Engineering**: I will perform further feature engineering to model various factors that, based on domain knowledge and insights from the Exploratory Data Analysis (EDA), are key predictors of default risk, I will then use RFE o slect an optimal feature set

2. **Pipeline Creation**: A pipeline will be created to generate these features from a reduced dataset with matching columns. This pipeline will be utilized on the `application_test_reduced` dataset and can be hypothetically applied to new incoming data.



## Feature Engineering Process

1. **Iniital Hypotheses**: I will formulate hypotheses based on insights from EDA as to what features could improve predictive power

2. **Automate transformations**: I will utilise tools like sklearn pipelines to handle transformations sequentially, with modular steps for scaling and feature creation. This is vital for point 3...

3. **Batch test created features**: I will be creating and evaluating features in batches to manage complexity and track any impact of created features and transformations, these features will be grouped by hypothesis, these features will be evaluated against simple baseline models.

4. **Assessing Robustness**: The robustness of selected features will be tested across a variety of different models and data splits and feature sets, cross validation wll used to assess the models ability to generalise.

5. **Final Selection**: Confirm a final high-quality feature set. Finalise the selection with techniques like RFE and SHAP






## Model Training and Evaluation Strategy
When comparing model performance, I will follow this strategy:

1. **Initial Training and Testing**: Initially, I will train and test models using the `application_train` dataset for convenience. This approach will allow for iterative refinement, hyperparameter tuning, and analysis of feature importance.

2. **Final Model Training**: Once satisfactory performance is achieved on the training set, I will retrain the model using the entire `application_train` dataset to leverage all available data for enhanced performance.

3. **Preprocessing and Feature Engineering on Application Test**: I will apply the same preprocessing and feature engineering pipelines to the `application_test` dataset to ensure consistency in feature representation.

4. **Scoring Metrics**: After applying the trained model to the `application_test` set, I will calculate scoring metrics (such as accuracy, precision, recall, F1-score, AUC, etc.) to evaluate how well the model generalizes to unseen data.

This structured approach will help ensure a thorough and systematic feature engineering process, leading to robust model performance.

In [1]:
#importing and reducing dfs
import pandas as pd
ap_df = pd.read_csv("application_train_train.csv")
cr_df = pd.read_csv("credit_card_balance.csv")
ap_val_df = pd.read_csv("application_train_val.csv")

from pipelines import application_cleaning_pipeline, application_encoding_pipeline, application_reduction_pipeline
from pipelines import credit_aggregation_pipeline, credit_cleaning_pipeline, credit_encoding_pipeline, credit_reduction_pipeline


### train set for hypotheses

ap_df = application_cleaning_pipeline.fit_transform(ap_df)
ap_df = application_encoding_pipeline.fit_transform(ap_df)
ap_df = pd.concat([application_reduction_pipeline.named_steps['top_feature_selector'].transform(ap_df),
                        application_reduction_pipeline.named_steps['aggregator'].transform(ap_df)], axis=1)


### validation set for hypotheses


ap_val_df = application_cleaning_pipeline.fit_transform(ap_val_df)
ap_val_df = application_encoding_pipeline.fit_transform(ap_val_df)
ap_val_df = pd.concat([application_reduction_pipeline.named_steps['top_feature_selector'].transform(ap_val_df),
                       application_reduction_pipeline.named_steps['aggregator'].transform(ap_val_df)], axis=1)


### additional information from credit set

cr_df = credit_aggregation_pipeline.fit_transform(cr_df)
cr_df = credit_cleaning_pipeline.fit_transform(cr_df)
cr_df = credit_encoding_pipeline.fit_transform(cr_df)
cr_df = pd.concat([
    credit_reduction_pipeline.named_steps['feature_selector'].transform(cr_df),
    credit_reduction_pipeline.named_steps['payment_behavior_aggregator'].transform(cr_df) 
], axis=1)



### function to merge application data with credit data in chunks
def merge_application_with_credit(application_df, credit_df, chunk_size=10000):
    merged_result = pd.DataFrame()
    for i in range(0, len(application_df), chunk_size):
        # Process each chunk and merge with credit data
        chunk = application_df.iloc[i:i+chunk_size]
        merged_chunk = pd.merge(chunk, credit_df, on='SK_ID_CURR', how='inner')
        merged_result = pd.concat([merged_result, merged_chunk], ignore_index=True)
    return merged_result

#Merge both application train and validation data with credit data
merged_train_df = merge_application_with_credit(ap_df, cr_df)
merged_val_df = merge_application_with_credit(ap_val_df, cr_df)


c:\Users\david\Projects\Credit_Risk_Project\pipelines.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna('Unknown', inplace=True)
c:\Users\david\Projects\Credit_Risk_Project\pipelines.py:116: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[

In [2]:
for col in merged_train_df.columns:
    print(col)

TARGET
SK_ID_CURR
EXT_SOURCE_2
EXT_SOURCE_3
DAYS_BIRTH
DAYS_ID_PUBLISH
DAYS_REGISTRATION
AMT_ANNUITY
DAYS_EMPLOYED
DAYS_LAST_PHONE_CHANGE
AMT_CREDIT
REGION_POPULATION_RELATIVE
AMT_INCOME_TOTAL
AMT_GOODS_PRICE
EXT_SOURCE_1
HOUR_APPR_PROCESS_START
AMT_REQ_CREDIT_BUREAU_YEAR
OWN_CAR_AGE
LIVINGAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
OBS_30_CNT_SOCIAL_CIRCLE
APARTMENTS_AVG
OBS_60_CNT_SOCIAL_CIRCLE
CNT_FAM_MEMBERS
COMMONAREA_AVG
YEARS_BUILD_AVG
NAME_EDUCATION_TYPE
NONLIVINGAREA_AVG
LANDAREA_AVG
BASEMENTAREA_AVG
LIVINGAPARTMENTS_AVG
CNT_CHILDREN
NAME_FAMILY_STATUS_Married
CODE_GENDER
FLAG_OWN_REALTY
TOTAL_DOCUMENT_FLAGS
STABILITY_INDEX
CONTACT_INDEX
CREDIT_BUREAU_REQ_TOTAL
SOCIAL_CIRCLE_OBS_TOTAL
SOCIAL_CIRCLE_DEF_TOTAL
AMT_PAYMENT_CURRENT_mean
AMT_DRAWINGS_OTHER_CURRENT_sum
CNT_DRAWINGS_ATM_CURRENT_sum
AMT_TOTAL_RECEIVABLE_mean
AMT_BALANCE_mean
AMT_CREDIT_LIMIT_ACTUAL_mean
SK_ID_PREV_count
AMT_INST_MIN_REGULARITY_mean
AMT_RECIVABLE_mean
AMT_RECEIVABLE_PRINCIPAL_mean
AMT_RECIVABLE_sum
AMT_RECEIV

# Hypotheses from EDA
### 1. Hypothesis on Skewness and Feature Transformation

**Hypothesis**: *Right-skewed financial features may reduce model stability and predictive accuracy. Applying log transformations to these skewed variables will improve distribution normality, enabling models to better capture relationships.*

**Rationale**:
- Financial metrics like income, credit balance, and receivables show heavy right-skew due to high-value outliers.
- Skewness can reduce performance in models that assume or benefit from Gaussian-like distributions (e.g., linear models, some tree-based algorithms), potentially introducing bias and reducing interpretability.

**Approach**:
- Apply log transformations to skewed financial features and analyze the resulting distributions.
- Assess model performance on both transformed and untransformed data, measuring predictive accuracy and stability across validation sets.

----

### 2. Hypothesis on Non-linear Relationship between Income and Default Rates

**Hypothesis**: *Income does not have a linear relationship with default probability; rather, higher incomes only start correlating with reduced default rates beyond the 75th percentile. Using polynomial features will help capture this non-linear relationship.*

**Rationale**:
- EDA revealed that income levels up to the 75th percentile show a relatively uniform default rate, suggesting that default likelihood remains high for lower income levels.
- Income only begins to meaningfully predict reduced default rates for values above this threshold, indicating a non-linear relationship.
- Introducing polynomial terms for income could allow the model to account for this non-linear effect, making it sensitive to higher incomes while maintaining accuracy for lower ranges.

**Approach**:
- Generate polynomial features for income and evaluate their predictive power against baseline models with linear income terms.
- Analyze the impact of the polynomial features on model performance, specifically regarding predictive accuracy for higher-income applicants.
- In addition to this there may be predicitve power in binning applicants based on income, this binned feature could then be interacted with other features to capture more granularity in the data

----
### 3. Hypothesis on Financial Stability and Wealth (particularly for Lower Credit applications)

**Hypothesis**: *Individuals purchasing lower-value properties, possibly attempting to stay within perceived financial limits, are likely of lower income and may face a higher risk of default due to lower wealth accumulation and financial stability.*

**Rationale**:
- EDA indicates a spike in default rates for individuals purchasing properties within the 25-50% price range and in the 25-50% credit range as these two are intrinsically tied together, suggesting these individuals may lack the financial flexibility to absorb unexpected expenses.
- Lower-value properties are typically associated with buyers from lower-income brackets, who may experience financial constraints that increase vulnerability to financial shocks.

#### Selected Features for Financial Vulnerability and Stability:

- **Income / Annuity**  
   - **Purpose**: Assesses the income proportion committed to annuity payments.  
   - **Hypothesis**: A lower ratio suggests a higher financial burden, indicating less flexibility to absorb unexpected expenses.

- **Income / Days Birth**  
   - **Purpose**: Proxies for lifetime earning potential relative to age.  
   - **Hypothesis**: Lower values imply under-earning for one’s age, indicating limited wealth accumulation and resilience.

- **Days Employed / Days Birth**  
   - **Purpose**: Measures employment stability relative to age.  
   - **Hypothesis**: A lower ratio indicates shorter employment durations, signaling possible job instability and financial vulnerability.

- **Current Payments Mean / Income**  
   - **Purpose**: Reflects the debt load in relation to income.  
   - **Hypothesis**: Higher values suggest limited disposable income, making individuals more susceptible to financial shocks.

- **Days Employed * Income**  
   - **Purpose**: Captures steady income over an extended employment history.  
   - **Hypothesis**: Higher values indicate stability and wealth accumulation potential, highlighting individuals with greater resilience even in lower income brackets.

These features aim to identify financially vulnerable individuals by modeling the interactions between income, employment stability, and financial commitments, providing insight into their resilience against financial shocks.

----

### 4. Hypothesis on "Overextension" as a key determinant of default risk across all incomes

**Hypothesis**: *Overextension financially, loans large relative to income and existing financial strain are key predictors of default*

**Rationale**:
 In the loan application data, defaulters often show loan annuities that are less proportional to their income, suggesting overextension. Financially overextended applicants may be at a higher risk of default, particularly if they lack income flexibility for unexpected expenses. To try and model this I will be looking at credit utilisation in addition to how factors like annuity and requested credit relate to an applicants income.


#### Selected Features to Model Overextension (Loan Application Only):

1. **Annuity-to-Income Ratio**  
   - **Purpose**: Represents the percentage of income dedicated to loan repayments.  
   - **Hypothesis**: Higher ratios indicate overextension, as a greater portion of income is allocated to debt servicing, reducing flexibility for other financial needs.

2. **Credit-to-Income Ratio**  
   - **Purpose**: Measures the requested loan amount relative to income.  
   - **Hypothesis**: Higher values suggest a heavier loan burden, indicating that applicants may be overextending themselves relative to their earning capacity.

3. **Annuity-to-Credit Ratio**  
   - **Purpose**: Reflects the loan terms in relation to the loan size.  
   - **Hypothesis**: Lower ratios might indicate longer loan terms with smaller payments, potentially chosen to accommodate affordability, signaling overextension if the applicant struggles with larger debts.

4. **Current Payments Mean / Income**  
   - **Purpose**: Assesses how much income is allocated toward servicing debt (from payments mean).  
   - **Hypothesis**: Higher values indicate limited financial flexibility, as more of the applicant’s income is used to service debt, elevating default risk if other expenses arise.

These features aim to capture overextension solely based on loan size, repayment structure, and income. An applicant exhibiting high values across these metrics is likely at increased risk for default due to overextension within the loan application context, without needing credit card limit data.

---

### 5. Hypothesis on Deviation from Group Norms

**Hypothesis**: *Individuals whose financial behaviors significantly deviate from the mean within their categorical groups (e.g., education level, gender, marital status) may exhibit distinct default risk profiles. Calculating deviations from group means for financial metrics will capture these atypical behaviors, enhancing the model’s ability to detect high-risk applicants.*

**Rationale**:
- In our EDA few categorical features were informative alone, just knowing marital status doesn't tell you much, however is this due to a lack of granularity in the original features?
- Group means for financial metrics within each category (e.g., average income within each education level) can serve as a "norm" or baseline.
- Individuals who significantly diverge from these norms (e.g., low income within a high-education group) may exhibit different levels of financial stability or risk tolerance than others in their group.
- Deviations from these group means can reveal patterns that aren’t captured by categories alone, allowing for a more granular assessment of financial behavior and risk.

**Approach**:
- Create mean difference features for groups
- Additioanlly interaction features will be created, interacting features like education level, gender, marital status on key numerical features like income, credit limits and so on.

---



#  Batch Testing Features

### Method
- Use a simple logistic regression model to see the impact of partiuclar transformations and created features on model performance for each given hypothesis
- Additionally so some features don't overwhelm others given that log reg is sensetive to scale I will be scaling the original dataset, and scaling after any created features or transformations

### Evaluation metrics
- I will include a range of metrics to evaluate created features on, however the most important features I will be tryng to improve upon here are Recall and AUC score, low precision isn't a massive concern at this stage as ideally through labelling and the meta model, these scores can be improved dramatically.


## Creating an Evaluation Function free from data leakage
This simple base pipeline can be created and reused for various hypothesis, it's simple it takes in a list of columns to scale and it scales them,

This will be used in conjunction with evaluation function, the evaluation function will take, a df, it will split the data, a list of transformations to do or features to create which will also be functions, and then the columns to scale. On each split the respective transformations and scaling is applied.

This approach is aimed to prevent data leakage by scaling and creating features exclusively on the train/validation set

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd



# Scaling function
def scale_columns(df, columns_to_scale, scaler=StandardScaler()):
    df = df.copy()  # Avoid mutating the original DataFrame
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    return df, scaler  # Return scaler to apply it on test set



# Evaluation function with transformations and scaling
def evaluate_log_reg_metrics(data, target_column="TARGET", index_column="SK_ID_CURR", 
                             columns_to_scale=None, transformations=None):
    # Separate features and target, excluding the index column
    X = data.drop(columns=[target_column, index_column])
    y = data[target_column]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
   
    # Apply transformations to both train and test sets separately
    if transformations:
        for transform in transformations:
            X_train = transform(X_train)
            X_test = transform(X_test)
    
    # Scale specified columns in train set and apply same scaler to test set
    if columns_to_scale:
        X_train, scaler = scale_columns(X_train, columns_to_scale)
        X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])  # Apply scaler from train to test

    # Define logistic regression model with balanced class weights
    model = LogisticRegression(class_weight="balanced", random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    
    # Predict probabilities and labels
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    auc = roc_auc_score(y_test, y_pred_prob)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Create a DataFrame to display metrics
    results_df = pd.DataFrame({
        "Metric": ["AUC Score", "Precision Score", "Recall Score", "F1 Score"], 
        "Score": [auc, precision, recall, f1]
    })
    
    print("\nLogistic Regression Model Evaluation:")
    print(results_df.to_string(index=False))


    """ Example Usage

    evaluate_log_reg_metrics(data=df, target_column="TARGET", index_column="SK_ID_CURR",
                         columns_to_scale=columns_to_scale, transformations=transformations)


    """



# Hypothesis 1: "Correcting for Right Skew will improve model performance"

In [4]:

default_columns_to_scale = [
    'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH',
    'DAYS_REGISTRATION', 'AMT_ANNUITY', 'DAYS_EMPLOYED', 'DAYS_LAST_PHONE_CHANGE',
    'AMT_CREDIT', 'REGION_POPULATION_RELATIVE', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE',
    'EXT_SOURCE_1', 'HOUR_APPR_PROCESS_START', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
    'OWN_CAR_AGE', 'LIVINGAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 
    'OBS_30_CNT_SOCIAL_CIRCLE', 'APARTMENTS_AVG', 'OBS_60_CNT_SOCIAL_CIRCLE', 
    'CNT_FAM_MEMBERS', 'COMMONAREA_AVG', 'YEARS_BUILD_AVG', 'NONLIVINGAREA_AVG', 
    'LANDAREA_AVG', 'BASEMENTAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'CNT_CHILDREN',
    'TOTAL_DOCUMENT_FLAGS', 'STABILITY_INDEX', 'CONTACT_INDEX', 
    'CREDIT_BUREAU_REQ_TOTAL', 'SOCIAL_CIRCLE_OBS_TOTAL', 'SOCIAL_CIRCLE_DEF_TOTAL',
    'AMT_PAYMENT_CURRENT_mean', 'AMT_DRAWINGS_OTHER_CURRENT_sum', 
    'CNT_DRAWINGS_ATM_CURRENT_sum', 'AMT_TOTAL_RECEIVABLE_mean', 'AMT_BALANCE_mean',
    'AMT_CREDIT_LIMIT_ACTUAL_mean', 'SK_ID_PREV_count', 'AMT_INST_MIN_REGULARITY_mean', 
    'AMT_RECIVABLE_mean', 'AMT_RECEIVABLE_PRINCIPAL_mean', 'AMT_RECIVABLE_sum',
    'AMT_RECEIVABLE_PRINCIPAL_sum', 'CNT_INSTALMENT_MATURE_CUM_sum', 
    'MONTHS_BALANCE_max', 'SK_DPD_max', 'SK_DPD_DEF_max', 'Payment_Behavior_Index'
]

def correct_right_skew(df):
    import numpy as np

    # Right-skewed features identified in EDA
    right_skewed_columns = [
        'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 
        'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_PAYMENT_CURRENT_mean', 'AMT_DRAWINGS_OTHER_CURRENT_sum', 
        'CNT_DRAWINGS_ATM_CURRENT_sum', 'AMT_TOTAL_RECEIVABLE_mean', 'AMT_BALANCE_mean', 
        'AMT_CREDIT_LIMIT_ACTUAL_mean', 'AMT_INST_MIN_REGULARITY_mean', 'AMT_RECIVABLE_mean', 
        'AMT_RECEIVABLE_PRINCIPAL_mean', 'AMT_RECIVABLE_sum', 'AMT_RECEIVABLE_PRINCIPAL_sum', 
        'CNT_INSTALMENT_MATURE_CUM_sum']                    

    df = df.copy()
    for col in right_skewed_columns:
        df[col] = np.log1p(df[col])
    return df


h1_transforms = [correct_right_skew]

print(" \n evaluation with log transforms (scaled):")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=default_columns_to_scale, transformations=h1_transforms)
print("\n evaluation no log transforms (scaled):")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=default_columns_to_scale)
print("\n evaluation no log transforms (not scaled):")
evaluate_log_reg_metrics(merged_train_df)

 
 evaluation with log transforms (scaled):

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.745618
Precision Score 0.175703
   Recall Score 0.662197
       F1 Score 0.277718

 evaluation no log transforms (scaled):

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744523
Precision Score 0.175465
   Recall Score 0.668582
       F1 Score 0.277977

 evaluation no log transforms (not scaled):

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.662458
Precision Score 0.150269
   Recall Score 0.553001
       F1 Score 0.236321


c:\Users\david\Projects\Credit_Risk_Project\Credit_Risk_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Evaluation of Log Transformation and Scaling in Model Performance

Below is a performance comparison across three configurations: **log-transformed and scaled**, **scaled without log transformation**, and **no transformations**. The table illustrates gains in **AUC**, **Precision**, **Recall**, and **F1 Score** from each configuration relative to the preceding one.

#### Model Performance

| Metric          | Log-Transformed + Scaled | Scaled Only | No Transforms + Not Scaled | Gain (Log-Scaled vs. Scaled Only) | Gain (Scaled vs. No Transforms) |
|-----------------|--------------------------|-------------|----------------------------|------------------------------------|----------------------------------|
| **AUC Score**   | 0.7456                   | 0.7445      | 0.6625                     | +0.15%                             | +12.38%                          |
| **Precision**   | 0.1757                   | 0.1755      | 0.1503                     | +0.14%                             | +16.83%                          |
| **Recall**      | 0.6622                   | 0.6686      | 0.5530                     | -0.95%                             | +20.91%                          |
| **F1 Score**    | 0.2777                   | 0.2780      | 0.2363                     | -0.11%                             | +17.64%                          |

#### Key Observations

1. **AUC Score**:
   - **Scaling Only** and **Log-Scaled Transformation** both produced significant AUC gains over the untransformed dataset, with **log transformation** providing an additional 0.15% gain.
   - A notable 12.38% AUC improvement occurred from applying scaling to the untransformed data, underscoring scaling's importance for model convergence.

2. **Precision**:
   - **Precision** experienced minimal gains from log transformation, showing a slight 0.14% increase.
   - Compared to the untransformed dataset, scaling alone improved precision by 16.83%.

3. **Recall**:
   - Interestingly, recall decreased by 0.95% with the addition of log transformation, but scaling alone provided a substantial recall gain of 20.91% over the untransformed baseline.

4. **F1 Score**:
   - The F1 score was largely consistent across configurations, with a marginal 0.11% decrease with log transformation.
   - From the untransformed dataset, scaling improved F1 score by 17.64%, showing the balance achieved with scaling alone.

**Conclusion**:  
The results indicate that **scaling without log transformation** provides the most balanced performance gains across recall and F1 score, while **log transformation offers a modest boost in AUC**. These findings suggest that, for this model, scaling alone is sufficient to achieve optimal performance gains without compromising recall or F1 balance.

#### Hypothesis 1: "Correcting for Right Skew will improve model performance"

Based on this analysis, **Hypothesis 1 is only partially supported**. Correcting for right skew with log transformations slightly improves AUC but does not yield substantial gains in precision, recall, or F1 score beyond what scaling alone achieves. This suggests that, while addressing skewness may help in certain metrics, it does not significantly impact overall performance in this context.


# 2. Hypothesis on Non-linear Relationship between Income and Default Rates

In [5]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

#function to create features:
def non_linear_income_features(df):
    """
    Create income quartile-based features for default risk prediction.
    
    Parameters:
    - df: pd.DataFrame - The DataFrame to add features to. Requires columns for AMT_INCOME_TOTAL, AMT_CREDIT, etc.
    
    Returns:
    - df: pd.DataFrame - DataFrame with new income quartile-based features added.
    """
    # Create income quartile feature
    df['AMT_INCOME_QUARTILE'] = pd.qcut(df['AMT_INCOME_TOTAL'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    df['AMT_INCOME_QUARTILE'] = df['AMT_INCOME_QUARTILE'].map({'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4}).astype(float)

    # Interaction and derived features
    df['AMT_INCOME_TOTAL_squared'] = df['AMT_INCOME_TOTAL'] ** 2
    df['INCOME_BIN_CREDIT'] = df['AMT_INCOME_QUARTILE'] * df['AMT_CREDIT']
    df['INCOME_QUARTILE_ANNUITY_RATIO'] = df['AMT_INCOME_QUARTILE'] * df['AMT_ANNUITY']
    df['INCOME_QUARTILE_EMPLOYED_RATIO'] = df['AMT_INCOME_QUARTILE'] * df['DAYS_EMPLOYED']
    df['INCOME_QUARTILE_EXT_SOURCE2'] = df['AMT_INCOME_QUARTILE'] * df['EXT_SOURCE_2']
    df['INCOME_QUARTILE_EXT_SOURCE3'] = df['AMT_INCOME_QUARTILE'] * df['EXT_SOURCE_3']
    df['INCOME_QUARTILE_AGE'] = df['AMT_INCOME_QUARTILE'] * abs(df['DAYS_BIRTH'])
    df['INCOME_QUARTILE_DEBT_RATIO'] = df['AMT_INCOME_QUARTILE'] * (df['AMT_CREDIT'] / (df['AMT_INCOME_TOTAL'] + 1e-5))
    df['INCOME_CREDIT_HISTORY_STABILITY'] = df['AMT_INCOME_QUARTILE'] * (df['SK_DPD_max'] + df['SK_DPD_DEF_max'])
    df['INCOME_DEBT_BURDEN_INDEX'] = df['AMT_INCOME_QUARTILE'] * (df['AMT_ANNUITY'] + df['AMT_TOTAL_RECEIVABLE_mean']) / (df['AMT_INCOME_TOTAL'] + 1e-5)
    df['DEBT_TO_CREDIT_QUARTILE'] = df['AMT_INCOME_QUARTILE'] * (df['AMT_CREDIT'] / (df['AMT_ANNUITY'] + 1e-5))

    return df

# Updated list of columns to scale, including any new columns in nl_df
h2_features =  ['AMT_INCOME_TOTAL_squared', 'INCOME_BIN_CREDIT', 'INCOME_QUARTILE_ANNUITY_RATIO', 'INCOME_QUARTILE_EMPLOYED_RATIO',
    'INCOME_QUARTILE_EXT_SOURCE2', 'INCOME_QUARTILE_EXT_SOURCE3', 'INCOME_QUARTILE_AGE', 'INCOME_QUARTILE_DEBT_RATIO',
    'INCOME_CREDIT_HISTORY_STABILITY', 'INCOME_DEBT_BURDEN_INDEX', 'DEBT_TO_CREDIT_QUARTILE']
h2_columns_to_scale = default_columns_to_scale + h2_features
h2_transforms = [non_linear_income_features]



print(" \n evaluation with h2 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=h2_columns_to_scale, transformations=h2_transforms)

print(" \n evaluation without h2 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=default_columns_to_scale)

h2_transforms = [correct_right_skew, non_linear_income_features]
### Does log scaling improve created features
print(" \n evaluation with h2 features created after log scaling:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=h2_columns_to_scale, transformations=h2_transforms)

 
 evaluation with h2 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744515
Precision Score 0.177425
   Recall Score 0.670498
       F1 Score 0.280599
 
 evaluation without h2 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744523
Precision Score 0.175465
   Recall Score 0.668582
       F1 Score 0.277977
 
 evaluation with h2 features created after log scaling:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.745358
Precision Score 0.176772
   Recall Score 0.660920
       F1 Score 0.278938


## Evaluation of Hypothesis 2: Impact of Non-linear Relationship Features with Log Scaling

This section evaluates the impact of creating new non-linear relationship features, both with and without prior log scaling of specific variables. Hypothesis 2 assumes that adding non-linear features based on income quartiles will improve model performance by capturing nuanced relationships between income and default risk.

#### Model Performance Comparisons

| Evaluation Configuration                  | AUC Score | Precision | Recall   | F1 Score | Gain in AUC (%) | Gain in Precision (%) | Gain in Recall (%) | Gain in F1 Score (%) |
|-------------------------------------------|-----------|-----------|----------|----------|-----------------|-----------------------|--------------------|-----------------------|
| **Without H2 Features**                   | 0.744523  | 0.175465  | 0.668582 | 0.277977 | -               | -                     | -                  | -                     |
| **With H2 Features**                      | 0.744515  | 0.177425  | 0.670498 | 0.280599 | -0.001%         | +1.12%                | +0.29%             | +0.94%                |
| **With H2 Features After Log Scaling**    | 0.745358  | 0.176772  | 0.660920 | 0.278938 | +0.11%          | +0.74%                | -1.15%             | +0.35%                |

#### Key Observations

1. **AUC Score**:
   - Adding H2 features alone led to a modest reduction in AUC by -0.001%.
   - Applying log scaling before creating H2 features showed a modest gain in AUC, from 0.744523 (baseline) to 0.745358, representing a gain of +0.11%.

2. **Precision**:
   - Precision improved modestly when H2 features were added, with a gain of +1.12%.
   - Log scaling before creating H2 features provided a precision gain of +0.74% over the baseline.

3. **Recall**:
   - Recall saw a modest gain with H2 features alone (+0.29%).
   - After log scaling, recall decreased to 0.660920, reflecting a loss of -1.15% compared to the baseline.

4. **F1 Score**:
   - The F1 score improved with H2 features alone, showing a gain of +0.94%.
   - After log scaling, the F1 score landed at 0.278938, a modest gain of +0.35% over the baseline.

#### Conclusion

The addition of non-linear relationship features (Hypothesis 2) provided modest improvements across the performance metrics, with notable gains in precision and F1 score. Applying log scaling before creating H2 features modestly increased AUC but did not lead to significant improvements in other metrics. This suggests that while non-linear transformations based on income quartiles capture some additional predictive nuance, their impact remains modest, and further refinement or feature selection may enhance these features' effectiveness.


# Hypothesis 3: Modeling Wealth Accumulation and Stability

In [6]:

def create_wealth_features(df):
    """
    Creates new wealth-related features directly on the provided DataFrame.
    """
    # Create new features directly in the input DataFrame
    quartile_threshold = df['AMT_GOODS_PRICE'].quantile(0.25)
    df['binary_feature'] = (df['AMT_GOODS_PRICE'] <= quartile_threshold).astype(int)
    df['income_annuity_ratio'] = df['AMT_INCOME_TOTAL'] / df['AMT_ANNUITY']
    df['income_days_birth_ratio'] = df['AMT_INCOME_TOTAL'] / (-df['DAYS_BIRTH'])
    df['days_employed_days_birth_ratio'] = df['DAYS_EMPLOYED'] / (-df['DAYS_BIRTH'])
    df['current_payments_income_ratio'] = df['AMT_PAYMENT_CURRENT_mean'] / df['AMT_INCOME_TOTAL']
    df['days_employed_income_product'] = df['DAYS_EMPLOYED'] * df['AMT_INCOME_TOTAL']
    df['financial_flexibility_score'] = (df['AMT_INCOME_TOTAL'] - df['AMT_PAYMENT_CURRENT_mean']) / df['AMT_GOODS_PRICE']
    df['dependency_load_index'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
    return df


h3_features = [
    ### new stuff
    'Payment_Behavior_Index', 'financial_flexibility_score', 'dependency_load_index', 'days_employed_days_birth_ratio', 'income_annuity_ratio', 'income_days_birth_ratio',
    'current_payments_income_ratio', 'days_employed_income_product',
]
h3_columns_to_scale = default_columns_to_scale + h3_features
h3_transforms = [create_wealth_features]

print(" \n evaluation with h3 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=h3_columns_to_scale, transformations=h3_transforms)

print(" \n evaluation without h3 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=default_columns_to_scale)


 
 evaluation with h3 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.745991
Precision Score 0.177384
   Recall Score 0.671137
       F1 Score 0.280603
 
 evaluation without h3 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744523
Precision Score 0.175465
   Recall Score 0.668582
       F1 Score 0.277977


## Evaluation of Hypothesis 3: Impact of Feature Set H3 on Model Performance

### Model Performance Comparisons

| Evaluation Configuration  | AUC Score | Precision | Recall   | F1 Score | Gain in AUC (%) | Gain in Precision (%) | Gain in Recall (%) | Gain in F1 Score (%) |
|---------------------------|-----------|-----------|----------|----------|-----------------|-----------------------|--------------------|-----------------------|
| **Without H3 Features**   | 0.744523  | 0.175465  | 0.668582 | 0.277977 | -               | -                     | -                  | -                     |
| **With H3 Features**      | 0.745991  | 0.177384  | 0.671137 | 0.280603 | +0.20%          | +1.10%                | +0.38%             | +0.94%                |

### Key Observations

1. **AUC Score**:
   - Adding H3 features resulted in a slight increase in AUC by +0.20%, indicating a marginal improvement in the model's ability to distinguish between classes.

2. **Precision**:
   - Precision improved by +1.10%, suggesting that the H3 features slightly increased the model's ability to correctly predict positive cases relative to total predicted positives.

3. **Recall**:
   - Recall saw a small gain of +0.38%, meaning the model captured a slightly higher proportion of actual positives with the H3 features.

4. **F1 Score**:
   - The F1 score also improved by +0.94%, showing a balanced improvement in both precision and recall due to the added H3 features.

### Conclusion

The addition of H3 features led to modest gains across all performance metrics, with the most notable improvement in precision. This suggests that the H3 feature set added valuable predictive nuance, though the overall impact remains relatively moderate. Further refinement or expansion of feature engineering might yield stronger performance gains.


# Modelling Overextension

In [7]:
def overextension_features(df):
    """
    Adds overextension-related features to the given DataFrame in place.
    
    Parameters:
    df (pd.DataFrame): DataFrame to modify with overextension features.

    Returns:
    pd.DataFrame: The modified DataFrame with new features added.
    """
    # 1. Annuity-to-Income Ratio
    df['annuity_income_ratio'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    
    # 2. Credit-to-Income Ratio
    df['credit_income_ratio'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    
    # 3. Annuity-to-Credit Ratio
    df['annuity_credit_ratio'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    # 4. Current Payments Mean to Income Ratio
    df['current_payments_income_ratio'] = df['AMT_PAYMENT_CURRENT_mean'] / df['AMT_INCOME_TOTAL']
    
    # 5. Credit Utilization Ratio
    df['credit_utilization_ratio'] = df['AMT_BALANCE_mean'] / df['AMT_CREDIT_LIMIT_ACTUAL_mean']
    
    # 6. Debt-to-Asset Ratio
    df['debt_asset_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    
    return df



h4_features = ['annuity_income_ratio', 'credit_income_ratio', 'annuity_credit_ratio', 'current_payments_income_ratio', 'debt_asset_ratio', 'credit_utilization_ratio']
h4_columns_to_scale = default_columns_to_scale + h4_features
h4_transforms = [overextension_features]


print(" \n evaluation with h4 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=h4_columns_to_scale, transformations=h4_transforms)

print(" \n evaluation without h4 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=default_columns_to_scale)

 
 evaluation with h4 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.745935
Precision Score 0.177508
   Recall Score 0.662197
       F1 Score 0.279968
 
 evaluation without h4 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744523
Precision Score 0.175465
   Recall Score 0.668582
       F1 Score 0.277977


## Evaluation of Hypothesis 4: Impact of Additional Ratio Features on Model Performance


### Model Performance Comparisons

| Evaluation Configuration  | AUC Score | Precision | Recall   | F1 Score | Gain in AUC (%) | Gain in Precision (%) | Gain in Recall (%) | Gain in F1 Score (%) |
|---------------------------|-----------|-----------|----------|----------|-----------------|-----------------------|--------------------|-----------------------|
| **Without H4 Features**   | 0.744523  | 0.175465  | 0.668582 | 0.277977 | -               | -                     | -                  | -                     |
| **With H4 Features**      | 0.745935  | 0.177508  | 0.662197 | 0.279968 | +0.19%          | +1.16%                | -0.95%             | +0.72%                |

### Key Observations

1. **AUC Score**:
   - The addition of H4 features led to a slight increase in AUC by +0.19%, showing a small improvement in the model’s ability to discriminate between classes.

2. **Precision**:
   - Precision improved by +1.16%, indicating a modest improvement in the model’s ratio of true positives to predicted positives with the H4 features.

3. **Recall**:
   - Recall decreased slightly by -0.95%, meaning the model captured a marginally lower proportion of actual positives after adding H4 features.

4. **F1 Score**:
   - The F1 score showed a slight improvement of +0.72%, suggesting a balanced gain in both precision and recall with the added features.

### Conclusion

The inclusion of H4 features provided modest gains in AUC, precision, and F1 score, with a small trade-off in recall. These results suggest that the additional ratio features in H4 capture some useful patterns, particularly improving precision. Overall, the H4 features show potential for enhancing model performance, though further optimization may yield more substantial gains.


# Deviation from group

In [8]:
def group_deviation_features(df):
    """
    Adds group deviation features to the DataFrame. For each column in financial_columns,
    this function calculates the deviation of each value from the group mean based on each
    grouping variable in group_columns.

    Parameters:
    df (pd.DataFrame): The DataFrame to modify with new deviation features.
    financial_columns (list of str): List of column names to calculate deviations for.
    group_columns (list of str): List of columns to group by.

    Returns:
    pd.DataFrame: The modified DataFrame with added deviation features.
    """

    # List of financial columns to create group deviation features for
    financial_columns = ['AMT_GOODS_PRICE', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_CREDIT_LIMIT_ACTUAL_mean']

    # Grouping columns
    group_columns = ['NAME_EDUCATION_TYPE', 'CODE_GENDER', 'NAME_FAMILY_STATUS_Married']

    for group_col in group_columns:
        for col in financial_columns:
            group_mean = df.groupby(group_col)[col].transform('mean')
            df[f'{col}_deviation_from_{group_col}_mean'] = df[col] - group_mean
            
    return df


h5_features = [
    'AMT_GOODS_PRICE_deviation_from_NAME_EDUCATION_TYPE_mean',
    'AMT_ANNUITY_deviation_from_NAME_EDUCATION_TYPE_mean',
    'AMT_CREDIT_deviation_from_NAME_EDUCATION_TYPE_mean',
    'AMT_CREDIT_LIMIT_ACTUAL_mean_deviation_from_NAME_EDUCATION_TYPE_mean',
    'AMT_GOODS_PRICE_deviation_from_CODE_GENDER_mean',
    'AMT_ANNUITY_deviation_from_CODE_GENDER_mean',
    'AMT_CREDIT_deviation_from_CODE_GENDER_mean',
    'AMT_CREDIT_LIMIT_ACTUAL_mean_deviation_from_CODE_GENDER_mean',
    'AMT_GOODS_PRICE_deviation_from_NAME_FAMILY_STATUS_Married_mean',
    'AMT_ANNUITY_deviation_from_NAME_FAMILY_STATUS_Married_mean',
    'AMT_CREDIT_deviation_from_NAME_FAMILY_STATUS_Married_mean',
    'AMT_CREDIT_LIMIT_ACTUAL_mean_deviation_from_NAME_FAMILY_STATUS_Married_mean'
]
h5_columns_to_scale = default_columns_to_scale + h5_features
h5_transforms = [group_deviation_features]


print(" \n evaluation with h5 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=h5_columns_to_scale, transformations=h5_transforms)

print(" \n evaluation without h5 features:")
evaluate_log_reg_metrics(merged_train_df, columns_to_scale=default_columns_to_scale)


 
 evaluation with h5 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744772
Precision Score 0.175521
   Recall Score 0.666667
       F1 Score 0.277881
 
 evaluation without h5 features:

Logistic Regression Model Evaluation:
         Metric    Score
      AUC Score 0.744523
Precision Score 0.175465
   Recall Score 0.668582
       F1 Score 0.277977


## Evaluation of Hypothesis 5: Impact of Group Deviation Features on Model Performance

### Model Performance Comparisons

| Evaluation Configuration  | AUC Score | Precision | Recall   | F1 Score | Gain in AUC (%) | Gain in Precision (%) | Gain in Recall (%) | Gain in F1 Score (%) |
|---------------------------|-----------|-----------|----------|----------|-----------------|-----------------------|--------------------|-----------------------|
| **Without H5 Features**   | 0.744523  | 0.175465  | 0.668582 | 0.277977 | -               | -                     | -                  | -                     |
| **With H5 Features**      | 0.744772  | 0.175521  | 0.666667 | 0.277881 | +0.03%          | +0.03%                | -0.29%             | -0.03%                |

### Key Observations

1. **AUC Score**:
   - The inclusion of H5 features led to a slight increase in AUC by +0.03%, suggesting a negligible improvement in model discrimination.

2. **Precision**:
   - Precision saw a minor gain of +0.03%, indicating a marginally improved rate of true positives to predicted positives with H5 features.

3. **Recall**:
   - Recall decreased slightly by -0.29%, meaning the model captured a marginally lower proportion of actual positives after adding H5 features.

4. **F1 Score**:
   - The F1 score showed a negligible decrease of -0.03%, suggesting that the addition of H5 features had a minimal overall impact on balancing precision and recall.

### Conclusion

The addition of group deviation features (H5) led to minor changes across all metrics, with a negligible increase in AUC and precision but a slight decrease in recall and F1 score. This suggests that while the group deviation features capture some information about group-specific deviations, their impact on overall model performance is limited. Further experimentation with more granular grouping variables or additional feature engineering may be needed to enhance predictive power.


# PIPELINE

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

# Custom transformer for scaling columns
class ScaleColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_scale):
        self.columns_to_scale = columns_to_scale
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns_to_scale])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.columns_to_scale] = self.scaler.transform(X[self.columns_to_scale])
        return X

# Custom transformer for non-linear income features
class NonLinearIncomeFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        return non_linear_income_features(X)

# Custom transformer for wealth-related features
class WealthFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        return create_wealth_features(X)

# Custom transformer for overextension-related features
class OverextensionFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        return overextension_features(X)
    
class SetIndex(BaseEstimator, TransformerMixin):
    def __init__(self, index_column):
        self.index_column = index_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.set_index(self.index_column, inplace=True)
        return X

# Define the feature engineering pipeline
def create_feature_engineering_pipeline(columns_to_scale, index_column="SK_ID_CURR"):
    feature_pipeline = Pipeline([
        ('non_linear_income_features', NonLinearIncomeFeatures()),
        ('wealth_features', WealthFeatures()),
        ('overextension_features', OverextensionFeatures()),
        ('scaling', ScaleColumns(columns_to_scale=columns_to_scale)),
        ('set_index', SetIndex(index_column=index_column))
        
    ])
    return feature_pipeline


# Assuming you have a DataFrame `df` with the required columns
# columns_to_scale = ['your', 'column', 'names']  # specify the columns you want to scale
# pipeline = create_feature_engineering_pipeline(columns_to_scale=columns_to_scale)
# transformed_df = pipeline.fit_transform(df)


all_columns_to_scale = ['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'AMT_ANNUITY', 'DAYS_EMPLOYED', 'DAYS_LAST_PHONE_CHANGE', 'AMT_CREDIT', 'REGION_POPULATION_RELATIVE', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'EXT_SOURCE_1', 'HOUR_APPR_PROCESS_START', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'LIVINGAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'OBS_30_CNT_SOCIAL_CIRCLE', 'APARTMENTS_AVG', 'OBS_60_CNT_SOCIAL_CIRCLE', 'CNT_FAM_MEMBERS', 'COMMONAREA_AVG', 'YEARS_BUILD_AVG', 'NONLIVINGAREA_AVG', 'LANDAREA_AVG', 'BASEMENTAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'CNT_CHILDREN', 'TOTAL_DOCUMENT_FLAGS', 'STABILITY_INDEX', 'CONTACT_INDEX', 'CREDIT_BUREAU_REQ_TOTAL', 'SOCIAL_CIRCLE_OBS_TOTAL', 'SOCIAL_CIRCLE_DEF_TOTAL', 'AMT_PAYMENT_CURRENT_mean', 'AMT_DRAWINGS_OTHER_CURRENT_sum', 'CNT_DRAWINGS_ATM_CURRENT_sum', 'AMT_TOTAL_RECEIVABLE_mean', 'AMT_BALANCE_mean', 'AMT_CREDIT_LIMIT_ACTUAL_mean', 'SK_ID_PREV_count', 'AMT_INST_MIN_REGULARITY_mean', 'AMT_RECIVABLE_mean', 'AMT_RECEIVABLE_PRINCIPAL_mean', 'AMT_RECIVABLE_sum', 'AMT_RECEIVABLE_PRINCIPAL_sum', 'CNT_INSTALMENT_MATURE_CUM_sum', 'MONTHS_BALANCE_max', 'SK_DPD_max', 'SK_DPD_DEF_max', 'Payment_Behavior_Index', 'AMT_INCOME_TOTAL_squared', 'INCOME_BIN_CREDIT', 'INCOME_QUARTILE_ANNUITY_RATIO', 'INCOME_QUARTILE_EMPLOYED_RATIO', 'INCOME_QUARTILE_EXT_SOURCE2', 'INCOME_QUARTILE_EXT_SOURCE3', 'INCOME_QUARTILE_AGE', 'INCOME_QUARTILE_DEBT_RATIO', 'INCOME_CREDIT_HISTORY_STABILITY', 'INCOME_DEBT_BURDEN_INDEX', 'DEBT_TO_CREDIT_QUARTILE', 'Payment_Behavior_Index', 'financial_flexibility_score', 'dependency_load_index', 'days_employed_days_birth_ratio', 'income_annuity_ratio', 'income_days_birth_ratio', 'current_payments_income_ratio', 'days_employed_income_product', 'annuity_income_ratio', 'credit_income_ratio', 'annuity_credit_ratio', 'current_payments_income_ratio', 'debt_asset_ratio', 'credit_utilization_ratio']

FE_pipeline = create_feature_engineering_pipeline(all_columns_to_scale)

FE_merged_train_df = FE_pipeline.fit_transform(merged_train_df)
FE_merged_val_df = FE_pipeline.fit_transform(merged_val_df)


# Getting and Optimal Feature Set with RFE

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt

# Separate features and target in the train and validation sets
X_train = FE_merged_train_df.drop(columns=["TARGET"])
y_train = FE_merged_train_df["TARGET"]
X_val = FE_merged_val_df.drop(columns=["TARGET"])
y_val = FE_merged_val_df["TARGET"]

# Initialize model
model = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=42)

# Lists to store results
n_features_list = range(10, X_train.shape[1], 10)  # Testing every 10 features
precision_scores = []
recall_scores = []
f1_scores = []

# Function to evaluate precision, recall, and f1
def evaluate_precision_recall_f1(X_train, y_train, X_val, y_val):
    # Train logistic regression model
    model = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    return precision, recall, f1

# Loop over different numbers of features for RFE
for n_features in n_features_list:
    rfe = RFE(model, n_features_to_select=n_features)
    rfe.fit(X_train, y_train)
    
    # Select the features and prepare train and validation sets
    selected_features = X_train.columns[rfe.support_]
    X_train_selected = X_train[selected_features]
    X_val_selected = X_val[selected_features]
    
    # Evaluate model performance with selected features
    precision, recall, f1 = evaluate_precision_recall_f1(X_train_selected, y_train, X_val_selected, y_val)
    
    # Store the results
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

# Plot Precision, Recall, and F1 Score against the number of features
plt.figure(figsize=(12, 6))
plt.plot(n_features_list, precision_scores, label="Precision", marker='o')
plt.plot(n_features_list, recall_scores, label="Recall", marker='o')
plt.plot(n_features_list, f1_scores, label="F1 Score", marker='o')
plt.xlabel("Number of Selected Features")
plt.ylabel("Score")
plt.title("Precision, Recall, and F1 Score vs. Number of Selected Features")
plt.legend()
plt.grid()
plt.show()


KeyboardInterrupt: 

## Feature Selection Choice: 40 Features

After analyzing the Precision, Recall, and F1 Score trends across different numbers of selected features, I chose to use **40 features** for the following reasons:

1. **Maximizing Recall and F1 Score**:
   - The plot of Precision, Recall, and F1 Score against the number of features shows a slight uptick in Recall and F1 Score around the 40-feature mark. Although Recall and F1 initially plateau around 10-15 features, this secondary increase at 40 features suggests that additional predictive information is being captured, particularly for identifying positive cases.
   - Since my primary objective is to maximize the model’s ability to identify positives (high Recall) while maintaining a balanced performance (high F1 Score), selecting 40 features allows me to capture these small but meaningful improvements.

2. **Balancing Complexity and Performance Gains**:
   - While using fewer features (10-15) could simplify the model, the minor performance gain at 40 features is valuable in this context, as it could enhance the model's robustness without a substantial increase in complexity.
   - By selecting 40 features, I aim to achieve an optimized balance between model complexity and predictive power, capturing nuanced patterns that may not be as evident with fewer features.

3. **Focus on Small Gains**:
   - Although the improvements are incremental, the secondary spike in Recall and F1 at 40 features justifies the decision to prioritize every possible gain. This choice reflects an optimization approach that values incremental improvements in predictive performance, even if they come at the cost of slightly increased model complexity.

### Conclusion

In conclusion, selecting **40 features** provides an optimal configuration that balances precision and recall while capturing minor but valuable performance gains. This choice is particularly beneficial for applications where maximizing positive case identification (Recall) and maintaining a balanced performance (F1 Score) are critical to model success.


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import pandas as pd


X_train = FE_merged_train_df.drop(columns=["TARGET"])  # Replace `TARGET` with your actual target column name
y_train = FE_merged_train_df["TARGET"]
X_val = FE_merged_val_df.drop(columns=["TARGET"])
y_val = FE_merged_val_df["TARGET"]

# Initialize the model
model = LogisticRegression(class_weight="balanced", max_iter=3000, random_state=42)

# Set up RFE with the model 
n_features_to_select = 40 
rfe = RFE(model, n_features_to_select=n_features_to_select)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Extract the selected features
selected_features = X_train.columns[rfe.support_]



def evaluate_model_no_split(X_train, y_train, X_val, y_val):
    # Initialize and train logistic regression model
    model = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    y_pred = model.predict(X_val)
    
    # Calculate evaluation metrics
    auc = roc_auc_score(y_val, y_pred_prob)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Return results as a dictionary
    return {
        "AUC Score": auc,
        "Precision Score": precision,
        "Recall Score": recall,
        "F1 Score": f1
    }


# Prepare the training and validation sets for selected features
selected_train_df = FE_merged_train_df[selected_features.tolist() + ["TARGET"]]
selected_val_df = FE_merged_val_df[selected_features.tolist() + ["TARGET"]]

# Split features and target
X_train_selected = selected_train_df.drop(columns=["TARGET"])
y_train_selected = selected_train_df["TARGET"]
X_val_selected = selected_val_df.drop(columns=["TARGET"])
y_val_selected = selected_val_df["TARGET"]

# Evaluate model with selected features
print("Evaluation with RFE Selected Features:")
selected_features_results = evaluate_model_no_split(X_train_selected, y_train_selected, X_val_selected, y_val_selected)
print(pd.DataFrame(selected_features_results, index=["RFE Selected Features"]).to_string(index=False))

# Split features and target for all features
X_train_all = FE_merged_train_df.drop(columns=["TARGET"])
y_train_all = FE_merged_train_df["TARGET"]
X_val_all = FE_merged_val_df.drop(columns=["TARGET"])
y_val_all = FE_merged_val_df["TARGET"]

# Evaluate model with all features
print("\nEvaluation with All Features:")
all_features_results = evaluate_model_no_split(X_train_all, y_train_all, X_val_all, y_val_all)
print(pd.DataFrame(all_features_results, index=["All Features"]).to_string(index=False))

# Combine and display comparison
comparison_df = pd.DataFrame([selected_features_results, all_features_results], index=["RFE Selected Features", "All Features"])
print("\nComparison of Model Performance:")
print(comparison_df)

Evaluation with RFE Selected Features:
 AUC Score  Precision Score  Recall Score  F1 Score
  0.761797         0.180884      0.689687  0.286601

Evaluation with All Features:
 AUC Score  Precision Score  Recall Score  F1 Score
   0.76277         0.179713      0.684554  0.284689

Comparison of Model Performance:
                       AUC Score  Precision Score  Recall Score  F1 Score
RFE Selected Features   0.761797         0.180884      0.689687  0.286601
All Features            0.762770         0.179713      0.684554  0.284689


In [12]:
### new segment of pipeline, there will be the option to create all features, or just the rfe set
selected_features = [
    'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_ANNUITY', 'DAYS_EMPLOYED',
    'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'EXT_SOURCE_1',
    'AMT_REQ_CREDIT_BUREAU_YEAR', 'OBS_30_CNT_SOCIAL_CIRCLE',
    'OBS_60_CNT_SOCIAL_CIRCLE', 'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS_Married', 'CODE_GENDER', 'STABILITY_INDEX',
    'CREDIT_BUREAU_REQ_TOTAL', 'CNT_DRAWINGS_ATM_CURRENT_sum',
    'AMT_TOTAL_RECEIVABLE_mean', 'AMT_BALANCE_mean',
    'AMT_CREDIT_LIMIT_ACTUAL_mean', 'AMT_INST_MIN_REGULARITY_mean',
    'AMT_RECIVABLE_mean', 'AMT_RECEIVABLE_PRINCIPAL_mean',
    'AMT_RECIVABLE_sum', 'AMT_RECEIVABLE_PRINCIPAL_sum',
    'CNT_INSTALMENT_MATURE_CUM_sum', 'AMT_INCOME_QUARTILE',
    'AMT_INCOME_TOTAL_squared', 'INCOME_BIN_CREDIT',
    'INCOME_QUARTILE_ANNUITY_RATIO', 'INCOME_QUARTILE_EXT_SOURCE2',
    'INCOME_QUARTILE_EXT_SOURCE3', 'INCOME_QUARTILE_AGE',
    'DEBT_TO_CREDIT_QUARTILE', 'binary_feature', 'income_days_birth_ratio',
    'days_employed_days_birth_ratio', 'credit_income_ratio',
    'annuity_credit_ratio', 'credit_utilization_ratio', 'debt_asset_ratio', "TARGET"
]

# Custom transformer for selecting RFE-selected features
class RFEFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, selected_features):
        self.selected_features = selected_features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.selected_features]

# Define the feature engineering pipeline with RFE feature selection
def create_feature_engineering_pipeline_rfe(columns_to_scale, selected_features, index_column="SK_ID_CURR"):
    feature_pipeline_rfe = Pipeline([
        ('non_linear_income_features', NonLinearIncomeFeatures()),
        ('wealth_features', WealthFeatures()),
        ('overextension_features', OverextensionFeatures()),
        ('scaling', ScaleColumns(columns_to_scale=columns_to_scale)),
        ('set_index', SetIndex(index_column=index_column)),
        ('feature_selection', RFEFeatureSelector(selected_features=selected_features))  # RFE feature selection step
    ])
    return feature_pipeline_rfe

###this code can be found copied to the pipelines.py script

## Evaluation of Gains from RFE Selected Features vs. All Features

### Model Performance Comparisons

| Evaluation Configuration  | AUC Score | Precision | Recall   | F1 Score | Gain in AUC (%) | Gain in Precision (%) | Gain in Recall (%) | Gain in F1 Score (%) |
|---------------------------|-----------|-----------|----------|----------|-----------------|-----------------------|--------------------|-----------------------|
| **All Features**          | 0.762770  | 0.179713  | 0.684554 | 0.284689 | -               | -                     | -                  | -                     |
| **RFE Selected Features** | 0.761797  | 0.180884  | 0.689687 | 0.286601 | -0.13%          | +0.65%                | +0.74%             | +0.67%                |

### Key Observations

1. **AUC Score**:
   - The AUC with RFE-selected features saw a slight decrease of -0.13% compared to using all features, indicating a minimal reduction in the model's discrimination ability.

2. **Precision**:
   - Precision improved by +0.65% with RFE-selected features, suggesting that the model with selected features provides a marginally better rate of true positives among predicted positives.

3. **Recall**:
   - Recall increased by +0.74% with RFE-selected features, indicating a slightly higher proportion of actual positives captured when using the RFE-selected subset.

4. **F1 Score**:
   - The F1 score gained +0.67% with RFE-selected features, reflecting a modest improvement in balancing precision and recall compared to using all features.

### Conclusion

The model with RFE-selected features demonstrated small but meaningful gains in Precision, Recall, and F1 Score over the model using all features, while incurring only a minor reduction in AUC. These results suggest that the RFE-selected feature subset achieves comparable (or slightly improved) performance with reduced complexity. By focusing on a smaller, more relevant feature set, the model may benefit from improved interpretability and reduced risk of overfitting, with minimal trade-offs in performance.


| Metric          | RFE Selected Features | Baseline Scaled | Gain (RFE vs. Baseline Scaled) |
|-----------------|-----------------------|-----------------|-------------------------------|
| **AUC Score**   | 0.7618                | 0.7445          | +2.32%                        |
| **Precision**   | 0.1809                | 0.1755          | +3.07%                        |
| **Recall**      | 0.6897                | 0.6686          | +3.16%                        |
| **F1 Score**    | 0.2866                | 0.2780          | +3.09%                        |